In [1]:
!pip install transformers peft scikit-learn

In [2]:
# Instalar as bibliotecas necessárias (execute no notebook se ainda não tiver as bibliotecas)

# Importar bibliotecas necessárias
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.model_selection import train_test_split
import torch

# Dados de exemplo
textos = [
    "Eu adorei este filme!",
    "O produto é péssimo, não recomendo.",
    "Excelente qualidade e ótimo atendimento.",
    "Não gostei, esperava mais.",
    "Maravilhoso, superou minhas expectativas!",
    "Terrível experiência, não voltarei.",
    "Muito bom, gostei bastante.",
    "Não foi como eu esperava.",
    "Excepcional, cinco estrelas!",
    "Decepcionante, serviço ruim."
]
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # 1 = Positivo, 0 = Negativo

# Dividir os dados em treino e validação
train_texts, val_texts, train_labels, val_labels = train_test_split(textos, labels, test_size=0.2, random_state=42)

# Carregar tokenizer e modelo pré-treinado
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Configurar LoRA com target_modules especificado
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,   # Tarefa de classificação de sequência
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=['q_lin', 'k_lin', 'v_lin']  # Módulos alvo para LoRA
)

# Aplicar LoRA ao modelo
model = get_peft_model(model, peft_config)

# Verificar os parâmetros treináveis
model.print_trainable_parameters()



/home/anderson/miniconda3/envs/scoras_academy/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/anderson/miniconda3/envs/scoras_academy/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFun

trainable params: 813,314 || all params: 67,768,324 || trainable%: 1.2001


In [3]:
#Restante do código abaixo:

In [4]:
# Tokenização
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Criar Dataset
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

# Configurações de treinamento (corrigido)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy='epoch',
    save_strategy='epoch',  # Adicionado para corresponder à evaluation_strategy
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)

# Definir função de métrica
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.from_numpy(logits), dim=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Criar trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Treinar o modelo
trainer.train()

# Avaliar o modelo
eval_results = trainer.evaluate()
print(eval_results)


/home/anderson/miniconda3/envs/scoras_academy/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                              
 20%|██        | 4/20 [00:01<00:03,  4.24it/s]

{'eval_loss': 0.6851018667221069, 'eval_accuracy': 0.5, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_f1': 0.6666666666666666, 'eval_runtime': 0.0719, 'eval_samples_per_second': 27.805, 'eval_steps_per_second': 13.902, 'epoch': 1.0}


                                              
 40%|████      | 8/20 [00:02<00:02,  5.30it/s]

{'eval_loss': 0.6810216903686523, 'eval_accuracy': 0.5, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_f1': 0.6666666666666666, 'eval_runtime': 0.0562, 'eval_samples_per_second': 35.578, 'eval_steps_per_second': 17.789, 'epoch': 2.0}


 55%|█████▌    | 11/20 [00:02<00:01,  5.13it/s]

...

{'eval_loss': 0.675743818283081, 'eval_accuracy': 0.5, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_f1': 0.6666666666666666, 'eval_runtime': 0.0664, 'eval_samples_per_second': 30.099, 'eval_steps_per_second': 15.05, 'epoch': 5.0}


100%|██████████| 20/20 [00:05<00:00,  3.59it/s]


{'train_runtime': 5.5691, 'train_samples_per_second': 7.182, 'train_steps_per_second': 3.591, 'train_loss': 0.6817650318145752, 'epoch': 5.0}


100%|██████████| 1/1 [00:00<00:00, 400.41it/s]

{'eval_loss': 0.6851018667221069, 'eval_accuracy': 0.5, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_f1': 0.6666666666666666, 'eval_runtime': 0.0623, 'eval_samples_per_second': 32.117, 'eval_steps_per_second': 16.059, 'epoch': 5.0}


# Descrição do Output do Código de Treinamento

## 1. Progresso do treinamento:
O treinamento ocorreu em 5 épocas, com 20 passos de treinamento no total. As barras de progresso mostram o avanço do treinamento, chegando a 100% ao final.

## 2. Métricas de avaliação:
A cada época, o modelo foi avaliado no conjunto de validação. As métricas principais são:

- `eval_loss`: A perda (loss) no conjunto de validação, que variou entre 0.67 e 0.68.
- `eval_accuracy`: A acurácia manteve-se constante em 0.5 (50%) durante todo o treinamento.
- `eval_precision`: A precisão também se manteve em 0.5 (50%).
- `eval_recall`: O recall foi consistentemente 1.0 (100%).
- `eval_f1`: O F1-score permaneceu em 0.6667 (aproximadamente 66.67%).

## 3. Métricas de treinamento:
- `loss`: A perda (loss) no conjunto de treinamento variou, com o último valor reportado sendo 0.6917.
- `grad_norm`: A norma do gradiente, que indica a magnitude das atualizações dos pesos, variou durante o treinamento.

## 4. Informações de tempo e velocidade:
- O tempo total de treinamento foi de aproximadamente 3.85 segundos.
- O modelo processou cerca de 10.4 amostras por segundo durante o treinamento.
- Foram realizados aproximadamente 5.2 passos de treinamento por segundo.

## 5. Resultado final:
Após o treinamento, uma avaliação final foi realizada, mostrando resultados similares aos observados durante o treinamento.

## Observações:
- O modelo parece estar com dificuldades para aprender, já que a acurácia permaneceu em 50% durante todo o treinamento.
- O recall de 100% combinado com uma precisão de 50% sugere que o modelo pode estar prevendo sempre a mesma classe, independentemente da entrada.
- Pode ser necessário ajustar os hiperparâmetros, aumentar o tamanho do conjunto de dados ou revisar a arquitetura do modelo para melhorar o desempenho.